<a href="https://colab.research.google.com/github/confusedolive/CE706/blob/master/lab04/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
np.set_printoptions(precision=3)

In [71]:
jester_data = pd.read_csv(r'https://raw.githubusercontent.com/albanda/CE888/master/lab4-recommender/jester-data-1.csv', na_values=99)
jester_df = jester_data.copy()

'''number of users is 24983 , 100 jokes , values to be treated as
nans are 99, column 74 to be dropped as its the amount of jokes rated by user'''

#drop column which contains number of jokes rated by user
jester_df.drop(labels='74',inplace=True, axis=1)
print(jester_df)

#best and worst rated jokes
best_rating_joke = jester_df.mean().idxmax()
worst_rating_joke = jester_df.mean().idxmin()

print(jester_df.iloc[12761, 37])

       -7.82  8.79  -9.66  -8.16  -7.52  ...  99.22  -5.63  99.23  99.24  99.25
0       4.08 -0.29   6.36   4.37  -2.38  ...  -2.14   3.06   0.34  -4.32   1.07
1        NaN   NaN    NaN    NaN   9.03  ...    NaN    NaN    NaN    NaN    NaN
2        NaN  8.35    NaN    NaN   1.80  ...    NaN    NaN    NaN    NaN    NaN
3       8.50  4.61  -4.17  -5.39   1.36  ...   1.55   3.11   6.55   1.80   1.60
4      -6.17 -3.54   0.44  -8.50  -7.09  ...  -3.35   0.05  -9.08  -5.05  -3.45
...      ...   ...    ...    ...    ...  ...    ...    ...    ...    ...    ...
24977   0.44  7.43   9.08   2.33   3.20  ...   9.03   6.55   8.69   8.79   7.43
24978   9.13 -8.16   8.59   9.08   0.87  ...  -8.20  -7.23  -8.59   9.13   8.45
24979    NaN   NaN    NaN    NaN  -7.77  ...    NaN    NaN    NaN    NaN    NaN
24980    NaN   NaN    NaN    NaN  -9.71  ...    NaN    NaN    NaN    NaN    NaN
24981   2.43  2.67  -3.98   4.27  -2.28  ...    NaN    NaN    NaN    NaN    NaN

[24982 rows x 100 columns]
-5.97


In [7]:
#Helper function
''' Label 10% of the dataset cells that are not Nans as 99, validation set
Keep actual values of the cells so you can use them later'''

arr = jester_df.values
rated = np.where(~np.isnan(arr))


def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(~np.isnan(arr))
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  
  return new_data, (rated[0][idx], rated[1][idx])


new_arr, idx = replace(arr)
print(new_arr)
print(rated)

[[ 4.08 -0.29  6.36 ...  0.34 -4.32  1.07]
 [  nan   nan   nan ...   nan   nan   nan]
 [  nan  8.35   nan ...   nan   nan   nan]
 ...
 [  nan   nan   nan ...   nan   nan   nan]
 [  nan   nan   nan ...   nan   nan   nan]
 [ 2.43 99.   -3.98 ...   nan   nan   nan]]
(array([    0,     0,     0, ..., 24981, 24981, 24981]), array([ 0,  1,  2, ..., 69, 70, 86]))


In [9]:

user_ratings = new_arr
arr = jester_df.values

latent_user_preferences = np.random.random((user_ratings.shape[0], 5))
latent_item_features = np.random.random((user_ratings.shape[1], 5))


def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0004):
    #print(item_id)
    predicted_rating = predict_rating(user_id, item_id)
    err =  predicted_rating - rating
    #print(err)
    user_pref_values = latent_user_preferences[user_id]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    mse_history = []
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = arr[user_id, item_id]
                if not np.isnan(rating):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        #if (iteration % 10000) == 0:
        print('Iteration %d/%d:\tMSE=%.6f' % (iteration, iterations, mse))
        mse_history.append(mse)
    return mse_history


# values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
# comparison_data = pd.DataFrame(values)
# print(comparison_data)
sgd(10)


Iteration 0/10:	MSE=25.026114
Iteration 1/10:	MSE=24.498123
Iteration 2/10:	MSE=24.070436
Iteration 3/10:	MSE=23.575935
Iteration 4/10:	MSE=22.801752
Iteration 5/10:	MSE=21.263625
Iteration 6/10:	MSE=19.309714
Iteration 7/10:	MSE=18.200498
Iteration 8/10:	MSE=17.791976
Iteration 9/10:	MSE=17.606034


[25.02611432096978,
 24.49812349570876,
 24.07043564366498,
 23.575935492205026,
 22.80175151666691,
 21.263624988162235,
 19.309713538913773,
 18.200498375671053,
 17.791975546003982,
 17.606034354655712]

In [82]:
movies_items = pd.read_excel('movies_latent_factors.xlsx',sheet_name = 'Items', index_col='Movie ID')
movie_users = pd.read_excel('movies_latent_factors.xlsx', sheet_name = 'Users', index_col='User')

#print(movies_items.loc[movies_items['Factor2'].idxmin()])
#print(movies_items.loc[1422])

user_rating = movie_users.loc[174].values
movie_rating = movies_items.loc[153].values[1:]
print(user_rating.dot(movie_rating))

val = movie_users.loc[1882].values
test = {}
for x in movies_items.values:
  test[x[0]] = val.dot(x[1:])

sort_dict= dict(sorted(test.items(), key=lambda item: item[1])) 

print(sort_dict)

-0.4653923887787672
{'Dumb & Dumber (1994)': -0.6473393628703262, 'The Rock (1996)': -0.5110025228216027, 'Mrs. Doubtfire (1993)': -0.5067425605778044, 'Braveheart (1995)': -0.4788640386410249, 'True Lies (1994)': -0.4456564758621034, 'Meet the Parents (2000)': -0.4334000849118009, 'Ace Ventura: Pet Detective (1994)': -0.43173927595230915, 'The Mask (1994)': -0.36924586262948295, 'The Patriot (2000)': -0.36526434595714413, 'Die Hard: With a Vengeance (1995)': -0.3633980867288325, 'Clear and Present Danger (1994)': -0.3184852589884998, 'Twister (1996)': -0.3127957958848003, 'Speed (1994)': -0.30971542770450006, 'The Fugitive (1993)': -0.3001258244419097, 'Dances with Wolves (1990)': -0.2843621758733564, 'Stargate (1994)': -0.26272690097455115, 'Batman Forever (1995)': -0.25915706367369157, 'Erin Brockovich (2000)': -0.24027640537278822, 'Forrest Gump (1994)': -0.21208019319628085, 'Cast Away (2000)': -0.2064962763166341, 'Apollo 13 (1995)': -0.20078137169468904, "Charlie's Angels (2000)